In [1]:
from accelerate import Accelerator
from transformers import AdamW, AutoTokenizer, AutoModelForCausalLM, get_scheduler
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

In [2]:
# Initialize the accelerator
accelerator = Accelerator()

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

optimizer = AdamW(model.parameters(), lr=3e-5)

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/leczhang/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [3]:
# Load the dataset and create a dataloader
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


# Move the model to the device
dataloader, model, optimizer = accelerator.prepare(
    dataloader, model, optimizer
)

In [4]:
num_epochs = 3
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [ ]:
progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    for batch in dataloader:
        # Tokenize the input text
        input_text = batch["text"]
        input_ids = tokenizer(input_text, return_tensors="pt", padding=True)["input_ids"]

        # Move the input to the device
        input_ids = accelerator.prepare(input_ids)
        
        #outputs = model(**batch)
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [2]:
def training_function():
    # Initialize the accelerator
    accelerator = Accelerator()
    
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForCausalLM.from_pretrained("gpt2")
    
    tokenizer.pad_token = tokenizer.eos_token
    
    optimizer = AdamW(model.parameters(), lr=3e-5)
    
    # Load the dataset and create a dataloader
    dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
    
    
    # Move the model to the device
    dataloader, model, optimizer = accelerator.prepare(
        dataloader, model, optimizer
    )
    
    num_epochs = 3
    num_training_steps = num_epochs * len(dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )
    progress_bar = tqdm(range(num_training_steps))
    model.train()
    
    for epoch in range(num_epochs):
        for batch in dataloader:
            # Tokenize the input text
            input_text = batch["text"]
            input_ids = tokenizer(input_text, return_tensors="pt", padding=True)["input_ids"]
    
            # Move the input to the device
            input_ids = accelerator.prepare(input_ids)
            
            #outputs = model(**batch)
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            accelerator.backward(loss)
    
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

In [3]:
from accelerate import notebook_launcher

notebook_launcher(training_function, num_processes=3)

Launching training on 3 GPUs.


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/leczhang/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True

RuntimeError: An issue was found when launching the training: 

-- Process 1 terminated with the following error:
Traceback (most recent call last):
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/leczhang/.local/lib/python3.10/site-packages/accelerate/utils/launch.py", line 543, in __call__
    self.launcher(*args)
  File "/tmp/ipykernel_3253632/896271484.py", line 44, in training_function
    outputs = model(input_ids, labels=input_ids)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1156, in forward
    output = self._run_ddp_forward(*inputs, **kwargs)
  File "/home/leczhang/.local/lib/python3.10/site-packages/torch/nn/parallel/distributed.py", line 1110, in _run_ddp_forward
    return modu